In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L

In [2]:
def conv_bl (inp, filters, dropout, activation="relu", kernel_initializer="he_normal", trainable = False):
    con1 = Conv2D(filters, (3,3), activation= activation, kernel_initializer=kernel_initializer, padding="same",)(inp)
    d1 = Dropout(dropout)(con1)
    con1 = Conv2D(filters, (3,3), activation= activation, kernel_initializer=kernel_initializer,padding="same", trainable = trainable)(d1)
    return con1

def enc(inp, filters, dropout, activation="relu", kernel_initializer="he_normal", trainable = False):
    c1 = conv_bl(inp, filters, dropout, activation = activation, kernel_initializer= kernel_initializer)
    c2 = conv_bl(c1, filters, dropout, activation = activation, kernel_initializer= kernel_initializer, trainable = trainable)
    p1 = MaxPooling2D((2,2))(c2)
    return c2, p1

def dec(inp, skip, filters, dropout, activation="relu", kernel_initializer="he_normal", trainable = False):
    up1 = Conv2DTranspose(filters, (2,2), (2,2))(inp)
    s1 = concatenate([up1, skip])
    c1 = conv_bl(s1, filters, dropout, activation = activation, kernel_initializer= kernel_initializer, trainable = trainable)
    return c1

def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

In [3]:
## Define constants
IMG_HEIGHT = 256
IMG_WIDTH = 256
BANDS = 3
inp_path = "" 
model_path = ""

In [4]:
### Define architecture

## Input
inp = Input((IMG_HEIGHT, IMG_WIDTH, BANDS))

## encoder_block          ##Output after pooling
e1,p1 = enc(inp, 16, 0.2) ##128,128,16
e2, p2 = enc(p1, 32, 0.2)  ##64,64,32
e3, p3 = enc(p2, 64, 0.2)  ##32,32,64
e4, p4 = enc(p3, 128, 0.2, kernel_initializer=None) ##16,16,128
e5, p5 = enc(p4, 256, 0.1) ##8,8,256

## base block
b1 = conv_bl(p5, 512, 0.1) ##8,8,512

## decoder block
d1 = dec(b1,e5, 256, 0.1, )
d2 = dec(d1, e4, 128, 0.1, trainable = True)
d3 = dec(d2, e3, 64, 0.0,trainable = True)
d4 = dec(d3, e2, 32, 0.0, trainable = True)
d5 = dec(d4, e1, 16, 0.0, trainable= True)

## output block
o1 = conv_bl(d5, 8, 0.0, trainable  = True)
o2 = conv_bl(o1,2, 0.0, trainable = True)
o3 = Conv2D(1, (3,3), (1,1), activation = "sigmoid", padding= "same" )(o2)


In [5]:
### Define hyperparameters
EPOCHS = 500
lr=1e-4
batch_size = 16

In [6]:
### Define callbacks
# mc = ModelCheckpoint(model_path, monitor = "val_loss", save_best_only =True, mode = "min", verbose = 1 )
# es = EarlyStopping(monitor = "val_loss", min_delta = 0, patience = 5, verbose = 1, restore_best_weights = True)
# lrr = ReduceLROnPlateau(monitor= "val_loss", factor = 0.1, patience = 5, verbose = 1, min_delta = 1e-4)

In [7]:
unet = Model(inputs=[inp], outputs = [o3], name="Unet_for_ship")
unet.compile(optimizer = Adam(learning_rate=lr), loss="binary_crossentropy", metrics = ["accuracy"])


In [8]:
eff = EfficientNetV2L(input_shape=(256,256,3), weights = "imagenet", include_top = False)

In [29]:
lay1 = eff.layers[19]
lay1w = lay1.get_weights()
lay2 = eff.layers[4]
lay2w = lay2.get_weights()

<Conv2D name=block1d_project_conv, built=True>
(3, 3, 32, 32)
<Activation name=stem_activation, built=True>


In [60]:
lay3 = eff.layers[5]
lay3w = lay3.get_weights()

<BatchNormalization name=block2g_expand_bn, built=True>
(256,)


In [11]:
lay3w

[array([[[[ 5.67017943e-02,  2.98023224e-07,  4.04792354e-02, ...,
           -8.61587673e-02,  1.54565722e-02,  1.45732546e-02],
          [ 5.17788157e-03,  6.92903996e-07, -8.04226846e-04, ...,
           -7.28617609e-03, -2.59024650e-03,  7.19625503e-04],
          [ 5.23283109e-02, -4.73260880e-05, -5.91974705e-03, ...,
           -2.84566842e-02,  1.02970283e-02, -8.96794908e-03],
          ...,
          [-2.36193836e-02,  3.11806798e-06,  1.16565526e-02, ...,
           -3.40198912e-02,  8.16247053e-03,  3.09990048e-02],
          [ 1.02026254e-01,  2.56448984e-05, -2.73126900e-01, ...,
           -1.47194326e-01, -1.89421698e-02, -2.15113144e-02],
          [ 7.50539750e-02,  3.34382057e-05, -1.13834534e-02, ...,
            1.33211434e-01, -2.81029809e-02,  2.49272078e-01]],
 
         [[-3.74776870e-01,  1.27404928e-06,  1.85772572e-02, ...,
           -9.57177877e-02,  1.89654902e-03, -6.24528900e-03],
          [ 1.10417604e-02, -3.05473804e-07, -1.96400285e-03, ...,
     

In [34]:
# Assuming you want to use default bias weights (all zeros)
kernel_weights = lay1w[0]  # Extract kernel weights
input_shape = unet.input_shape[1:]  # Get input shape of the model's input tensor
output_channels = kernel_weights.shape[-1]  # Get the number of output channels

# Initialize bias weights with zeros
bias_weights = np.zeros((output_channels,))
ulay1 = unet.get_layer(index = 13)

# Set weights for ulay1
ulay1.set_weights([kernel_weights, bias_weights])



In [19]:
# # test_con =  Conv2D(32, (3,3), (1,1), activation = "relu", padding="same", weights =  [lay1w])(p3)
# test_con =  Conv2D(32, (3,3), (1,1), activation = "relu", padding="same", weights =  [lay1w])(p3)
# test_con.set_weights(lay1w)

In [14]:
unet.summary()

Model: "Unet_for_ship"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256, 256,  │          0 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      2,320 │ dropout[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 256, 256,  │      2,320 │ conv2d_1[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256, 256,  │          0 │ conv2d_2[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │      2,320 │ dropout_1[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128, 128,  │          0 │ conv2d_4[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 128,  │      9,248 │ dropout_2[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 128, 128,  │      9,248 │ conv2d_5[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128, 128,  │          0 │ conv2d_6[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 128, 128,  │      9,248 │ dropout_3[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 64, 64,    │          0 │ conv2d_8[0][0]  

 Total params: 9,349,851 (35.67 MB)

 Trainable params: 4,827,643 (18.42 MB)

 Non-trainable params: 4,522,208 (17.25 MB)

In [15]:
eff.summary()

Model: "efficientnetv2-l"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        864 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      9,216 │ stem_activation[… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │        128 │ block1a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 32)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      9,216 │ block1a_add[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │        128 │ block1b_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 32)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_co… │ (None, 128, 128,  │      9,216 │ block1b_add[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_bn  │ (None, 128, 128,  │        128 │ block1c_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_ac… │ (None, 128, 128,  │          0 │ block1c_project_

 Total params: 117,746,848 (449.17 MB)

 Trainable params: 117,234,272 (447.21 MB)

 Non-trainable params: 512,576 (1.96 MB)

In [16]:
### Define and compile model

unet = Model(inputs=[inp], outputs = [o3], name="Unet_for_ship")
unet.compile(optimizer = Adam(learning_rate=lr), loss="binary_crossentropy", metrics = ["accuracy"])


In [17]:
for layers in unet.layers:
    print(layers)

<InputLayer name=input_layer, built=True>
<Conv2D name=conv2d, built=True>
<Dropout name=dropout, built=True>
<Conv2D name=conv2d_1, built=True>
<Conv2D name=conv2d_2, built=True>
<Dropout name=dropout_1, built=True>
<Conv2D name=conv2d_3, built=True>
<MaxPooling2D name=max_pooling2d, built=True>
<Conv2D name=conv2d_4, built=True>
<Dropout name=dropout_2, built=True>
<Conv2D name=conv2d_5, built=True>
<Conv2D name=conv2d_6, built=True>
<Dropout name=dropout_3, built=True>
<Conv2D name=conv2d_7, built=True>
<MaxPooling2D name=max_pooling2d_1, built=True>
<Conv2D name=conv2d_8, built=True>
<Dropout name=dropout_4, built=True>
<Conv2D name=conv2d_9, built=True>
<Conv2D name=conv2d_10, built=True>
<Dropout name=dropout_5, built=True>
<Conv2D name=conv2d_11, built=True>
<MaxPooling2D name=max_pooling2d_2, built=True>
<Conv2D name=conv2d_12, built=True>
<Dropout name=dropout_6, built=True>
<Conv2D name=conv2d_13, built=True>
<Conv2D name=conv2d_14, built=True>
<Dropout name=dropout_7, built=